# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the predator-prey domain described in the Homework and which you described as a Markov decision process.

<img src="toroidal-world.png" width="400px">

Recall that:

* toroidal world "wraps around", i.e., an individual exiting through any of the four sides of the grid reenters on the opposite side (see figure above).

* At each time step, the hare selects uniformly at random one of the four directions (up, down, left, and right) and moves to the adjacent cell in that direction with a probability 0.4. With a probability 0.6 it remains in the same cell. 

* The wolf, on the other hand, can select at each time step one of five actions---up (_U_), down (_D_), left (_L_) and right (_R_) or stay (_S_). If it selects action _S_, it remains in the same cell with probability 1.0. Otherwise, the other 4 actions succeed in moving the wolf to the adjacent cell in the corresponding direction with a probability 0.8 and fail with a probability 0.2. 

* The goal of the wolf is to catch the hare.

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note 1**: Don't forget to import `numpy`.

**Note 2**: You can define the transition probability matrices for each of the two individuals and then build the combined transition probability matrices using the `numpy.kron` function.

---

In [334]:
import numpy as np
from pandas import *

X = [[0, 0], [0, 1], [0, 2], [0, 3],
     [1, 0], [1, 1], [1, 2], [1, 3], 
     [2, 0], [2, 1], [2, 2], [2, 3], 
     [3, 0], [3, 1], [3, 2], [3, 3]]

A = ['L', 'R', 'U', 'D', 'S']

P_coelho = np.array([[0.6, 0.2, 0.2, 0.0], 
					 [0.2, 0.6, 0.0, 0.2], 
					 [0.2, 0.0, 0.6, 0.2], 
					 [0.0, 0.2, 0.2, 0.6]])

P_lobo_rl = np.array([[0.2, 0.8, 0.0, 0.0], 
		     		  [0.8, 0.2, 0.0, 0.0], 
		     		  [0.0, 0.0, 0.2, 0.8], 
		     		  [0.0, 0.0, 0.8, 0.2]])
 
P_lobo_ud = np.array([[0.2, 0.0, 0.8, 0.0], 
			 		  [0.0, 0.2, 0.0, 0.8], 
		     		  [0.8, 0.0, 0.2, 0.0], 
		     		  [0.0, 0.8, 0.0, 0.2]])

P_lobo_s = np.array([[1.0, 0.0, 0.0, 0.0], 
					 [0.0, 1.0, 0.0, 0.0], 
		    		 [0.0, 0.0, 1.0, 0.0], 
		    	  	 [0.0, 0.0, 0.0, 1.0]])

C = np.array([[1.0,  1.0,  1.0,   1.0,  0.0], #(0,0)
			  [0.0,  0.0,  1.0,   1.0,  1.0], #(0,1)
			  [1.0,  1.0,  0.0,   0.0,  1.0], #(0,2)
			  [1.0,  1.0,  1.0,   1.0,  1.0], #(0,3)

			  [0.0,  0.0,  1.0,   1.0,  1.0], #(1,0)
			  [1.0,  1.0,  1.0,   1.0,  0.0], #(1,1)
			  [1.0,  1.0,  1.0,   1.0,  1.0], #(1,2)
			  [1.0,  1.0,  0.0,   0.0,  1.0], #(1,3)

			  [1.0,  1.0,  0.0,   0.0,  1.0], #(2,0)
			  [1.0,  1.0,  1.0,   1.0,  1.0], #(2,1)
			  [1.0,  1.0,  1.0,   1.0,  0.0], #(2,2)
			  [0.0,  0.0,  1.0,   1.0,  1.0], #(2,3)

			  [1.0,  1.0,  1.0,   1.0,  1.0], #(3,0)
		 	  [1.0,  1.0,  0.0,   0.0,  1.0], #(3,1)
			  [0.0,  0.0,  1.0,   1.0,  1.0], #(3,2)
			  [1.0,  1.0,  1.0,   1.0,  0.0]])#(3,3)

P_RL = np.kron(P_lobo_rl, P_coelho)
P_UD = np.kron(P_lobo_ud, P_coelho)
P_S = np.kron(P_lobo_s, P_coelho)

mdp = {"X": X, "A": A, "Pa's": [P_RL, P_RL, P_UD, P_UD, P_S], "C":C} 


### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $(w, h)$, always moves the wolf to the cell closest to the hare. If multiple such cells exist, the wolf should select randomly between the two.

For example, suppose that the wolf is in cell 1 and the hare in cell 4 (figure above, left). The wolf should then select randomly between the actions _U_, _D_ (which move the wolf to cell 3), _L_ and _R_ (which move the wolf to cell 2). Conversely, if the wolf is in cell 1 and the hare in cell 3 (figure above, right), the wolf should select randomly between the two actions _U_ and _D_ (which move the wolf to cell 3).

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [306]:
policy_smart =np.array([[0.0,  0.0,  0.0,  0.0,  1.0], # 00
						[0.5,  0.5,  0.0,  0.0,  0.0], # 01
						[0.0,  0.0,  0.5,  0.5,  0.0], # 02
						[0.25, 0.25, 0.25, 0.25, 0.0], # 03

						[0.5,  0.5,  0.0,  0.0,  0.0], # 10
						[0.0,  0.0,  0.0,  0.0,  1.0], # 11
						[0.25, 0.25, 0.25, 0.25, 0.0], # 12
						[0.0,  0.0,  0.5,  0.5,  0.0], # 13

						[0.0,  0.0,  0.5,  0.5,  0.0], # 20
						[0.25, 0.25, 0.25, 0.25, 0.0], # 21
						[0.0,  0.0,  0.0,  0.0,  1.0], # 22
						[0.5,  0.5,  0.0,  0.0,  0.0], # 23
                        
						[0.25, 0.25, 0.25, 0.25, 0.0], # 30
						[0.0,  0.0,  0.5,  0.5,  0.0], # 31
						[0.5,  0.5,  0.0,  0.0,  0.0], # 32
						[0.0,  0.0,  0.0,  0.0,  1.0]]); # 33



---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2. Use $\gamma=0.99$.

---

In [308]:
#@brief:
#   This function receives a matrix with costs and a policy and computes the costs for that policy.
def calculate_Cpi(costs, policy):
    cpi = np.zeros(len(policy))
   
    # loop over states
    for i in range(0, len(policy)):        
        # loop over actions to compute sum(pi(a|x) * c(a,x))

        sum = 0
        for j in range(0, len(costs[0])):
            sum += policy[i][j]*costs[i][j]

        cpi[i] = sum
    return cpi

#@brief:
#   This function receives a dictionary that represents a MDP and a policy and computes the transiction 
# matrix for that MDP with that policy. 
def transition_Probabilities_For_Policy(MDP, policy):
    Ppi = np.zeros((len(MDP["X"]),len(MDP["X"]))) # creates a matrix NxN where N is the size of X
    
    for i in range(0, len(MDP["X"])):
        for j in range(0, len(MDP["X"])):
            
            # for every entry we need to loop over possible actions to compute sum(pi(a|x)*P(y|x,a))
            sum = 0
            for k in range(0,len(MDP["A"])):
                sum += policy[i][k] * MDP["Pa's"][k][i][j]
            
            Ppi[i][j] = sum
    return Ppi

Cpi = calculate_Cpi(costs=C, policy=policy_smart)
Ppi = transition_Probabilities_For_Policy(MDP=mdp, policy=policy_smart)

gamma = 0.99
Jpi = np.dot(np.linalg.inv((np.identity(16)-(gamma*Ppi))), Cpi) # [I - gama* (Ppi)^-1] *Cpi = Jpi  

labels = ["00", "01", "02", "03", "10", "11", "12", "13", "20", "21", "22", "23", "30", "31", "32", "33"]
print (DataFrame(Jpi, index=labels, columns=["Jpi"]))


         Jpi
00  2.279619
01  2.337185
02  2.337185
03  3.675743
10  2.337185
11  2.279619
12  3.675743
13  2.337185
20  2.337185
21  3.675743
22  2.279619
23  2.337185
30  3.675743
31  2.337185
32  2.337185
33  2.279619


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is optimal: use value iteration to compute $J^*$ and show that $J^*=J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [309]:
def computeJnew(Qa):
    Jnew = np.zeros((len(Qa[0]),1))
    for i in range(0, len(Qa[0])):
        min = Qa[0][i]
        for j in range(1, len(Qa)):
            if Qa[j][i] < min:
                min = Qa[j][i]
        Jnew[i][0] = min
    return Jnew
    
def valueIteration(MDP, tolerance, gama):
    J = np.zeros((len(MDP["X"]), 1))
    err = 1
    i=0
    while err > tolerance:
        
        Qa = [None]*len(MDP["A"]) #initialize empty list for Q values for actions a in A
        
        # loop over actions and compute Qa
        for a in range(0, len(MDP["A"])):
            Qa[a] = MDP["C"][:,[a]] + gama * MDP["Pa's"][a].dot(J)
        
        Jnew = computeJnew(Qa)
        err = np.linalg.norm(Jnew - J)
        i += 1
        J = Jnew    
    return (J, i)

time_before = time.time()
J_optimal, iterations = valueIteration(MDP=mdp, tolerance=1e-8, gama=0.99)
time_after = time.time()

print (DataFrame(Jpi, index=labels, columns=["Jpi"]))
print (DataFrame(J_optimal.T[0], index=labels, columns=["J optimal"]))



         Jpi
00  2.279619
01  2.337185
02  2.337185
03  3.675743
10  2.337185
11  2.279619
12  3.675743
13  2.337185
20  2.337185
21  3.675743
22  2.279619
23  2.337185
30  3.675743
31  2.337185
32  2.337185
33  2.279619
    J optimal
00   2.279619
01   2.337185
02   2.337185
03   3.675743
10   2.337185
11   2.279619
12   3.675743
13   2.337185
20   2.337185
21   3.675743
22   2.279619
23   2.337185
30   3.675743
31   2.337185
32   2.337185
33   2.279619


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [310]:
def policyIteration(MDP):
    pi = np.ones((len(MDP['X']), len(MDP['A']))) /2
    quit = False
    i = 0
    
    while not quit:
        
        #initialize cpi with first line of pi multiplyed by the costs for the first action
        cpi = np.diag(pi[:,0]).dot(MDP["C"][:,[0]])
        ppi = np.diag(pi[:, 0]).dot(MDP["Pa's"][0])
        for a in range(1, len(MDP["A"])):
            cpi += np.diag(pi[:,a]).dot(MDP["C"][:,[a]])
            ppi += np.diag(pi[:, a]).dot(MDP["Pa's"][a])

        J = np.linalg.inv(np.eye(len(MDP["X"])) - gamma * ppi).dot(cpi)
        
        Qa = [None] * len(MDP["A"])
        
        for a in range(0, len(MDP["A"])):
            Qa[a] = MDP["C"][:,[a]] + gamma * MDP["Pa's"][a].dot(J)
        
        pinew = np.zeros((len(MDP['X']), len(MDP['A'])))
        
        for a in range(0, len(MDP["A"])):
            pinew[:, a, None] = np.isclose(Qa[a], np.min(Qa, axis=0), atol=1e-8, rtol=1e-8).astype(int)

        pinew = pinew / np.sum(pinew, axis=1, keepdims=True)
        quit = (pi == pinew).all()
        pi = pinew
        i +=1
    return (pi, i)

pi, iterations = policyIteration(MDP=MDP)
print (pi)
print (iterations)

[[ 0.000  0.000  0.000  0.000  1.000]
 [ 0.500  0.500  0.000  0.000  0.000]
 [ 0.000  0.000  0.500  0.500  0.000]
 [ 0.250  0.250  0.250  0.250  0.000]
 [ 0.500  0.500  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  1.000]
 [ 0.250  0.250  0.250  0.250  0.000]
 [ 0.000  0.000  0.500  0.500  0.000]
 [ 0.000  0.000  0.500  0.500  0.000]
 [ 0.250  0.250  0.250  0.250  0.000]
 [ 0.000  0.000  0.000  0.000  1.000]
 [ 0.500  0.500  0.000  0.000  0.000]
 [ 0.250  0.250  0.250  0.250  0.000]
 [ 0.000  0.000  0.500  0.500  0.000]
 [ 0.500  0.500  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  1.000]]
2


### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting in each of the two states $x$ in the initial figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [333]:
s0 = 12 # position corresponding to state 30 in X 
s1 = 7  # position corresponding to state 20 in X
        
def generateRandomTrajectory(MDP, policy, s0, steps=10000):
    trajectory = np.array([(s0, None)])
    
    for i in range(1, steps):
        print (policy[trajectory[i-1][0]])
        random_action = np.random.choice(len(MDP['A']), size=1, p=policy[trajectory[i-1][0]])
        possible_state = np.random.choice(len(MDP['X']), size=1, p=MDP["Pa's"][random_action][trajectory[i-1][0]])
        
        trajectory = np.append(trajectory, [(possible_state, random_action)])
    return trajectory

trajectory =  generateRandomTrajectory(mdp, pi, 12, steps=10)
print (trajectory)



[ 0.250  0.250  0.250  0.250  0.000]


/Users/ricardorei/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


TypeError: 'NoneType' object is not subscriptable